In [2]:
import carball
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch
import torch.nn as nn
import torch.optim as optim
%matplotlib inline
cuda = True

In [3]:
from data import Calculated

data_manager = Calculated()

In [15]:
num_columns = 36
class HitPredictor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_columns, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.Sigmoid()
        )
    
    def forward(self, game_state):
        return self.layers(game_state)

In [16]:
categories = ['pass_', 'passed', 'dribble', 'dribble_continuation', 'shot', 'goal', 'assist', 'assisted', 'save', 'aerial']
def get_train_test_data(proto, dataframe):
    # Only from the perspective of the blue team (we'll switch them up)
#     players = dataframe.columns.levels[0]
#     columns = dataframe.columns.levels[1]
    players = proto.players
    teams = proto.teams
    team_map = {}
    for i, team in enumerate(teams):
        for player in team.player_ids:
            team_map[player.id] = int(team.is_orange)
            
    name_map = {}
    for i, player in enumerate(players):
        name_map[player.id.id] = player.name
    
    hits = proto.game_stats.hits
    inputs = []
    outputs = []
    for hit in hits:
        player_name = name_map[hit.player_id.id]
        
        frame = dataframe.iloc[hit.frame_number]
        
        player_location = frame[player_name]
        ball_location = frame['ball']
        input_vector = ball_location.values
        input_vector = np.append(input_vector, player_location.values)
        
        inputs.append(input_vector)
        vector = np.zeros(len(categories))
        for i, c in enumerate(categories):
            if getattr(hit, c):
                vector[i] = 1
        outputs.append(vector)
    inputs = np.array(inputs).astype(float)
    outputs = np.array(outputs)
    rand = np.random.rand(outputs.shape[0])
    threshold = rand < 0.8
    input_train, input_test = inputs[threshold], inputs[~threshold]
    output_train, output_test = outputs[threshold], outputs[~threshold]
    
    return input_train, input_test, output_train, output_test


loss = nn.BCELoss()
# loss = nn.MSELoss()
def train_on_proto_df(proto, df, model):
    input_train, input_test, output_train, output_test = get_train_test_data(proto, df)
    if input_train is None:
        print('data is none')
        return
    input_tensor = torch.from_numpy(input_train).float()
    output_tensor = torch.from_numpy(output_train).float()
    if cuda:
        input_tensor = input_tensor.cuda()
        output_tensor = output_tensor.cuda()
#     print(output_train.shape, output_test.shape)
    print(proto.game_metadata.match_guid)
    for x in range(1000):
        opt.zero_grad()
        predicted = model(input_tensor)
#         print("shapes", predicted.shape, output_tensor.shape)
        loss_val = loss(predicted, output_tensor)
        loss_val.backward()
        opt.step()
        if x % 30 == 0:
#             print(input_tensor.shape, predicted.shape, output_tensor.shape)
#             print('Loss:', loss_val)
            input_test_tensor = torch.from_numpy(input_test).float()
            if cuda:
                input_test_tensor = input_test_tensor.cuda()
            test_output = model(input_test_tensor)
            output = test_output.cpu().detach().numpy()
            accuracy = (output.round() == output_test)
    return loss_val.cpu().data.numpy(), accuracy

In [ ]:
model = HitPredictor().train()
if cuda:
    model = model.cuda()
opt = optim.Adam(model.parameters(), lr=1e-4)
epochs = 25
epoch_losses = []
accuracy = []
for epoch in range(epochs):
    epoch_loss = []
    epoch_accuracy = []
    for n in range(50):
        for replay in data_manager.get_replay_list(num=100, page=n + 1):
            df = data_manager.get_pandas(replay)
            proto = data_manager.get_proto(replay)
            if df is None or proto is None:
                continue
            try:
                los, acc = train_on_proto_df(proto, df, model)
            except Exception as e:
                print(e)
                continue
            epoch_loss.append(los)
            epoch_accuracy.append(acc)
    epoch_losses.append(epoch_loss)
    accuracy.append(epoch_accuracy)

93516E0A11E8CB5641B484834097B559
single positional indexer is out-of-bounds
053BCA4A11E8CB5958BEA9A2198661CE
8D713E6A11E8CB57A709D3AB31A9144E
55C0B91811E8CB5817755F956D034FCF
3407A73E11E8CB5BB03AD0AE8F6C9447
20D26C4A11E8CB5AE1479B9F97E7C31F
285660FA11E8CB5C9C26F1A7286FAD60
8A67334611E8CB5C574301B00E97E74B
08FA2AF211E8CB61A3B237A6E5965C19
B5ED32DE11E8CB634EA412AF95DACF62
450E02B211E8CB6F208C2F87736BCBEC
A834A77A11E8CBC34B9AE1A23EC41588
6EFC793811E8CBCD4D7A869AF8FC2425
A7148CF211E8CBD2B57A58B7E67B0A36
FCE2844C11E8CBD51F3C92907B43BB89
A42A8A2211E8CBC473BB98B448F85957
8593AC0A11E8CBCFD2DC45ACA58FAB1B
9725DEF811E8CBD31EA75BB6664E36BA
4F3995B611E8CBD96A5595A686B9BEA2
F3E9297011E8CBDC177579806D034FCF
A7B5D82A11E8CBD552882481C25E20E3
078361BC11E8CBD8035AE49AB9C3C4DE
7B41F57411E8CBE68ACD90AB7597FDD4
55B1AD1411E8CBF4F91618A46E3E0D8C
440B025A11E8CBEEA045BEA675A4E62F
1320BECC11E8CBFE4D7A4CA6F8FC2425
4222021A11E8CC00208C9A99736BCBEC
5852314811E8CC0217F2A089B32AFB08
D978473011E8CC0CD42E10A4A9813356


13AF652011E8DE26386394B6893AE1C5
BEB4117011E8DE293EFF4CA4942F1B69
C6EE0B0011E8DE2B17754CA06D034FCF
700CAD9A11E8DE3B24956392D234FDAF
FB7E9B4011E8DE3BDBCA90A3676B6A04
648CCCB411E8DE3D754B01A127B6CAA2
75DA4D8A11E8DE3CEDF1A2A438FAFC7A
A4443BB011E8DE44035A98A1B9C3C4DE
C09333A011E8CFD20A94A19EE9134521
6BAFA42A11E8DE48AC1312B1CBF30AF2
D617D92A11E8DE4F889B6883F4BC8AE2
0F476ACE11E8DE9347A2BE8AD253EF33
2B317D9411E8DE96AF9F1ABF8770A65E
426D761C11E8DE950B3C889906FF7F13
C51988C611E8DEB3CA89CAB3B66CE599
B5622F1811E8DEE107748694DB1F9E4B
4646799411E8DEE247B6F291D3AEA3D3
B75DBB3C11E8DEE3F2492EB4A76F5C8B
38DEFF1C11E8DEE538B7DC9693F5F544
BFC6D77611E8DEE441B47B974097B559
13D69B5211E8DEF5AC1344A3CBF30AF2
C1E3ECE011E8DEFF6D96D6896EC3989C
A93BCE9E11E8DF029BD842BAB6670E51
994354CC11E8DF10F424B3B3EE4D099D
A71BB2CA11E8DF1429D8AD9D326A0091
831AA9F411E8DF1323239F8430B9C58C
DD8EA6A411E8DF1547A25D8ED253EF33
BED4FE5811E8E77A025449B5DDA89CDC
A0721E7811E8E77AAC13CEA3CBF30AF2
180EFC2611E8E77BAEF45CA858E7A53D
94D7FC8011

BB747C7611E8AD1D177569B26D034FCF
71B197FE11E88C2566EB45909DB5566D
B61A9AA211E829665CEB87A590E3C53A
E23ABA3411E8E2DB9D2E06A3805781BF
414042C411E8E2F0B5C41ABE82CFBEB3
458566BA11E8E2F60CA58889835BDE8F
B9FFA6CC11E8E2F1A1279A895CEED59D
5D39FBB211E8E2F7D67A50B978296A08
1927E17A11E8E30473BB26BF48F85957
46BB849011E8E307D95178BBBDA20AF6
4784A99611E8E3084E04589FD445D2A4
217E9F9411E8E3096D964C956EC3989C
471E9DF011E8E30C7E2F3EA959D821EA
E3317CB211E8E3205288E7AEC25E20E3
944F819811E8E31B1B1CF9B3FD072066
769A7DBC11E8E373F175A3A9284E6B98
77463A8211E8E38AD2DC8090A58FAB1B
75D5D4D411E8D023C23CBB8ACFA9171F
2DB682A211E8E3926940DE9FA37BC12B
F8CEEF7411E8E3920A9480B0E9134521
AF385C9E11E8E3955CEB69B190E3C53A
AF746B0411E8E394C204A9BB025A91F2
single positional indexer is out-of-bounds
4A10988E11E8E39C8F563DB86E14AA3E
C6E3938211E8E3A8FB16EB9482A3D4C9
D0F11B7211E8E3AA4A3BFF9B21291C79
AB21B8DA11E8E3A9F0ECE6AC6EE3B5AE
4F40B99211E8E3ABF5C4E594BCD7229C
100A965011E8E3AE8F56579F6E14AA3E
9F8F8F2811E8E3AFC79C48A312999F9A


3F68DABE11E8CF466750D1A8F2D2FDCA
092B70F411E8DA56CCE02B99B78C084D
5F7FEB0011E8DA5C177539AB6D034FCF
94FCD30811E8DA640B3CDE9F06FF7F13
29C2369A11E8DAA1D45C8E91D0C6BE61
18BE1E8011E8DAA26CAF01BB5D03616E
C0BAB6FC11E8DABB99EF6EABA9D8C6B3
0F31C45011E8DAC2B03A48BD8F6C9447
4F95ADA211E8DAC452886D94C25E20E3
56A37AF811E8DAC8573AF999682D8D8B
057886FC11E8DAD5E47AD2A39535D834
D9C5EA6211E8DADA01C5A4B803FEA074
EB7C6F8C11E8DAEAF424BAADEE4D099D
626E500611E8DAEB5E19079487866E61
F9CAA57611E8DAEBE06E53A7DF9F6172
5AA41C3C11E8DAEDF24979B2A76F5C8B
26E0BCA211E8DAED87EF21AB777CAF14
3E7097D611E8DAF0B57AFEB1E67B0A36
3E25AFD811E8DAEF595336AA60D4B411
C40E3B9211E8DAEF17F212A2B32AFB08
09A1228A11E8DAF2CA89C4BDB66CE599
2C4EEA7811E8DAF3CF595B903158549E
A528293C11E8DAF3A64B178FD39CEC1D
A0EA732011E8DAF87A7976B210B8E4A9
B8D4D17011E8DAF787EF40A2777CAF14
B3C1D49811E8DAF8B55ED7B77C7D2EAD
single positional indexer is out-of-bounds
A5C0887A11E8DAF9999ECAA3EB511BAF
B36EBA2611E8DAFB574377940E97E74B
CB34238011E8DAFB958402BD2C9F5CBB


81CE748811E8CA8FAC1326BFCBF30AF2
6CCB0C6A11E8CD8001F23A9787D06070
2E043E1A11E8CA989A1F299B066AC2C0
7764BF4611E8CA9B36D1239D0FF9EBAE
BA64E57A11E8CA9911E9BA9593B724A9
3017512211E8CA9FD42E10A5A9813356
42B47D4811E8CA9D5DB51F96C37FF21A
F001FF7011E8CAA7D9E730872995E0DA
DE37165211E8CAA901F2B3B987D06070
40B59E3611E8CAB72CC8BABF2D90E774
B3A3808411E8CDA08E8065B48982EB9B
1A82FA6611E8CADEEDAC5BA7D8DA96A8
5C07F62C11E8CDA397F01AB4BA7C9F3F
1B9BF23A11E8CDA506B01DAACFA29187
229F537811E8CDA638634A97893AE1C5
52C37D0411E8CB0DC11EBBA05E677929
4A1839DC11E8CB09D67AF9A378296A08
B22CBA5C11E8CB1384246FAF72A1FFC1
157181D811E8CDA89D2EB79B805781BF
82A4466E11E8CB147B42D1906A3A4A60
9B9720EA11E8CB16E06E9980DF9F6172
52DF280811E8CB15C8453090C5A4EE2F
19D3D31011E8CB1588C0ECA0A264708C
83C9F00C11E8CB19FB939CBEF4D6E9E4
351F227011E8CB1D699684AEEA63DF0F
F975EB0211E8CB1F7EF4E3B78BFB2691
46E28ACC11E8CB1F1C8D2CACDDA7BA9C
08A2F47611E8CB20F055DBA2D16595B0
EB51F02C11E8CB1D2A6C08A6A4BFED02
C0568D1611E8CB210EA52AB3FB96DBC2
9A0EA14C11

...

7D890F6411E8C72ECC417FAB6B37926C
D96785F011E8C737039CC592EF58DF76
2AFDE3CC11E8C734B2DF36B885912911
BBD3B63211E8C73F451A07860A8916A6
079C8D7C11E8C480F36DD9B795A970F4
5065B27611E8B7B66F928EBE15A8A97B
DCE5882011E8C74CA12710965CEED59D
6B7C372011E8C750DE5E1EB48C50DEBA
6C31DA2A11E8C7515E19F29E87866E61
23A3951011E8C75548B08E8871BE58A5
C0D7775211E8C7559A1F5CBC066AC2C0
C35F464A11E8C754386326AD893AE1C5
1EDD32C411E8C756F249B181A76F5C8B
331A776411E8C75757C1DBBD71517278
102826C011E8D35ACAF872B99C9B171C
11B1B9DC11E8C76341B40EB14097B559
6B939DCE11E8C76A460A159E61C00416
6F0BD7C611E8D360958401B82C9F5CBB
03C09A9611E8D3614A767EBE1E040DB9
723574E411E8D46DDDD3779F6FA86103
C095894A11E8C7EC4EA4D58095DACF62
D63069C211E8C7ED52DA2C95600D04F5
624C0ECA11E8C7EE3FB998A492002E75
EDCA727E11E8C7EFEF450BB27FA4632F
C8B1F06411E8C7F19F723F93DDB876BB
BFD401C211E8C7F09945B1BFA3A4312F
6A6D85B211E8C7FC9584B8982C9F5CBB
A694BFF411E8C7FE4F670EAA0180EC12
46C0895A11E8C7FE15AB67A17925C4DE
29833F5811E8C7FF60EFDB874B79BA5F
C60C46FC11

Failure to read pandas [] from memory: Struct error
Traceback (most recent call last):
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/numpy_manager.py", line 49, in get_array
    starting_byte = struct.unpack('i', chunk)[0]
struct.error: unpack requires a buffer of 4 bytes

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/pandas_manager.py", line 27, in safe_read_pandas_to_memory
    return PandasManager.read_numpy_from_memory(buffer)
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/pandas_manager.py", line 40, in read_numpy_from_memory
    array = read_array_from_file(buffer)
  File "/home/matthew/PycharmProjects/ReplayML/venv/lib/python3.6/site-packages/carball/analysis/utils/numpy_manager.py", line 36, in read_array_fro

0C169B0E11E8CF113FBD92A13AC558AA
7D26081A11E8E5C4FB16828E82A3D4C9
7E31A75611E8E5C30BA8428680B43590
77133B5411E8E5D4DBCAA884676B6A04
695170FA11E8E5D7A24703A69539603F
BB16E49E11E8E5DF41B403BA4097B559
9780807611E8E5E415AB7B877925C4DE
5344BB8E11E8E5E98424839A72A1FFC1
103B123011E8E5ED99C273995776FF5B
F431EAE811E8E5EFAAB63794AABD8C0B
20BAACA211E8E5FB5589169CF4CBF7D5
8D1487F611E8E6196CAF3DB65D03616E
809B99CC11E8E61BFC4E32A3FA80569B
8F6AA42611E8E61AE99333B9E1517F09
00D42C9C11E8E6231A4AA9801FBB4684
ECF467E011E8E6238E6CA9A5E0470387
EC9B980611E8E6262AEEB9B36E4F2067
D0AA67C811E8E629B03ABFA68F6C9447
646B48B411E8E630396D01A868244BC6
88CDDE3C11E8E631035AC99BB9C3C4DE
CE22B1CA11E8E63A9D2EF393805781BF
FB61ED4E11E8E63BA24770A39539603F
F8AE053C11E8E63C195B7181825AC0B0
073340E611E8E63CE993D998E1517F09
8F1434E011E8E63F07E5EC90EC8E432E
F7FDB0FC11E8E640396D04A468244BC6
2FE84A1A11E8E6498E801F848982EB9B
D86770E611E8E647C1B55CB2510D7945
B5DE978211E8E6491D11C9A6A35E3316
02D7CECC11E8E696811A138FED9FF7A4
690431C011

B7FC6FC611E8D4C955896892F4CBF7D5
single positional indexer is out-of-bounds
9259FFF411E8C9BBFB9AB5AC4F41F148
7D973FEE11E8C9B8B2DB4CBD2D537FA2
75134BCC11E8C9BA01C5B78003FEA074
70A0204011E8C9BC69408992A37BC12B
single positional indexer is out-of-bounds
F5FD979011E8C9D036C0059066B5B600
single positional indexer is out-of-bounds
424D257211E8C9D4445ABC916F5EF3F1
F0CE763A11E8D3FE2CC8A2B12D90E774
4B39295E11E8D4AD66146F830729A5D6
8C5393F411E8D401CA8983A1B66CE599
9381AF6A11E8D4AD2CC833BA2D90E774
BD1242E411E8D404D951B7AFBDA20AF6
B032ABD411E8D4060C9C0FAB93EEA1E6
80C01D0411E8D407A86FCD867AC938C8
B50CEE2E11E8CA08F424C986EE4D099D
B0EF0FC411E8D40982679285A47CA8FD
65348FDC11E8CA09E9934E96E1517F09
B698BB9A11E8CA0F15E01AA5E1E2E2C4
3AB4322211E8CA124A3BF3B421291C79
D22F8CF611E8D40C84BB359B1A6F09FC
DAF57CE211E8CA20A4D5679F381B22F2
E76694E811E8CA251AC65FB56E42E181
FC66739211E8D40FD42E39BBA9813356
9204640011E8CA28460AD4AE61C00416
0F41B7DE11E8CA2D60EF90A84B79BA5F
210F5B1611E8CA2CD67AD2AC78296A08
A461E15411E8C

EC4B4B5A11E8D1A2F5C414BFBCD7229C
152E0FC411E8D1A55288E29EC25E20E3
022CD8B411E8D1A6EDF110BB38FAFC7A
E59785FE11E8D1A638637E90893AE1C5
B7DFB8F411E8D1AEA283F598692A63D5
5E81B21611E8D1B4DE5E5F9C8C50DEBA
5CA5C1F211E8D1B57A565AABDFA85F6B
CB60451C11E8D1B6A709948B31A9144E
C5BC54E211E8D1B769960599EA63DF0F
7F034DE011E8D1CA30F865B8613B6737
98D84B0011E8D1CD15E0E8A2E1E2E2C4
58B9BC6C11E8D1F9688E1391EA6BEC4F
61BBAFCA11E8D20BF36DE48C95A970F4
AEFBA68611E8D20C9C26C193286FAD60
F9CDBEBA11E8D2116E05339163E6D63F
3677D91211E8D2409A1F4BBC066AC2C0
69C62BE811E8D240CCE0A9A7B78C084D
0DBAEAC211E8C1933F943A97513C40BD
3D49142211E8D3625345BB8867A4072F
A677ECBA11E8D36317D1998F0871F2A1
0EF2C05411E8C2C460EFB1A14B79BA5F
841AF31E11E8C6DAE286839324E68ECC
C2AEE33611E8C6E14D7AFF9FF8FC2425
single positional indexer is out-of-bounds
5A3A150011E8C6F01FAB20B518155A90
BBE8C98211E8C6F40FD7BBB81B760370
DDFB51FA11E8C6F6DE5E0FBD8C50DEBA
C601F33811E8C700DDD358A76FA86103
EB3CE14811E8C7011545F4A0FBA4FD2F
26D0EC9211E8C70084240DA872A1FFC1


In [ ]:
means = []
for l in epoch_losses:
    l = np.array(l)
    l = l[l != None]
    means.append(l.mean())

plt.plot(range(len(epoch_losses)), means)
plt.title('Loss')

In [ ]:
# [epoch#, replay#, hit#, category#]
means = []
for epoch in accuracy:
    epoch = np.asarray(np.asarray(epoch).tolist())
    replay_means = []
    for replay in epoch:
        cat_means = []
        for i, c in enumerate(categories):
            cat_means.append(np.array(replay[:, i].mean()))
        replay_means.append(np.array(cat_means))
    means.append(np.array(replay_means))
means = np.array(means)

for i, c in enumerate(categories):
    data = [means[epoch][:, i].mean() for epoch in range(len(accuracy))]
    plt.plot(data)
plt.legend(categories)
# plt.plot(range(len(accuracy)), means)
# plt.title('Accuracy')
#plt.legend(['Team Accuracy', 'Frame Accuracy']);